In [1]:
%load_ext autoreload
%autoreload 2

# Get parent directory and add to sys.path
import os
import sys

parent_dir = os.path.dirname(os.getcwd())
sys.path.append(parent_dir)

# Require ipympl
%matplotlib widget 

In [2]:
# MPC import
import numpy as np
from MPCVelControl_D3_2 import MPCVelControl

from MPCControl_xvel_D3_2 import MPCControl_xvel
from MPCControl_xvel_D3_2 import MPCControl_xvel_tuned_default
from MPCControl_xvel_D3_2 import MPCControl_xvel_tuned_final

from MPCControl_yvel_D3_2 import MPCControl_yvel
from MPCControl_yvel_D3_2 import MPCControl_yvel_tuned_default
from MPCControl_yvel_D3_2 import MPCControl_yvel_tuned_final

from MPCControl_zvel_D3_2 import MPCControl_zvel
from MPCControl_zvel_D3_2 import MPCControl_zvel_tuned_default
from MPCControl_zvel_D3_2 import MPCControl_zvel_tuned_final

from MPCControl_roll_D3_2 import MPCControl_roll
from MPCControl_roll_D3_2 import MPCControl_roll_tuned_default
from MPCControl_roll_D3_2 import MPCControl_roll_tuned_final


from src.rocket import Rocket
from src.vel_rocket_vis import RocketVis

rocket_obj_path = os.path.join(parent_dir, "Cartoon_rocket.obj")
rocket_params_path = os.path.join(parent_dir, "rocket.yaml")

In [ ]:
Ts = 0.05
sim_time = 7
H = 5.0
N = int(H / Ts)

rocket = Rocket(Ts=Ts, model_params_filepath=rocket_params_path)
xs, us = rocket.trim()
A, B = rocket.linearize(xs, us)

x0 = xs.copy()

x_target = xs.copy()
x_target[6] = 3.0                 # vx
#x_target[7] = 3.0                 # vy
#x_target[8] = 3.0                 # vz
#x_target[5] = np.deg2rad(35.0)    # gamma (roll angle)


rocket = Rocket(Ts=Ts, model_params_filepath=rocket_params_path)
mpc = MPCVelControl().new_controller(rocket, Ts, H)

t_cl, x_cl, u_cl, t_ol, x_ol, u_ol, _ = rocket.simulate_control(
    mpc, sim_time, H, x0, x_target=x_target, method="linear"
)

vis = RocketVis(rocket, rocket_obj_path)
vis.anim_rate = 1.0
vis.animate(
    t_cl[:-1],
    x_cl[:, :-1],
    u_cl,
    Ref=x_target.reshape(-1, 1).repeat(u_cl.shape[1], axis=1),
    T_ol=t_ol[..., :-1],
    X_ol=x_ol,
    U_ol=u_ol,
)

x_target.reshape(-1, 1).repeat(u_cl.shape[1], axis=1)


******************************************************************************
This program contains Ipopt, a library for large-scale nonlinear optimization.
 Ipopt is released as open source code under the Eclipse Public License (EPL).
         For more information visit https://github.com/coin-or/Ipopt
******************************************************************************

Simulating time 0.00: Iteration 1... not yet converged

Iteration 2... not yet converged

Iteration 3... not yet converged

Iteration 4... not yet converged

Iteration 5... not yet converged

Iteration 6... not yet converged

Iteration 7... not yet converged

Maximum invariant set successfully computed after 8 iterations.
Iteration 1... not yet converged

Iteration 2... not yet converged

Iteration 3... not yet converged

Iteration 4... not yet converged

Iteration 5... not yet converged

Iteration 6... not yet converged

Iteration 7... not yet converged

Maximum invariant set successfully computed after 8

RuntimeError: steady_state_from_ref failed: infeasible

In [ ]:
#Open loop plots

# ---------- initial conditions ----------
x_target_x = xs.copy(); x_target_x[6] += 3.0
x_target_y = xs.copy(); x_target_y[7] += 3.0
x_target_z = xs.copy(); x_target_z[8] += 3.0
x_target_r = xs.copy(); x_target_r[5]  = np.deg2rad(35.0)
